In [3]:
import os
import json
import kagglehub
from edgar import *
from tqdm import tqdm
from kagglehub import KaggleDatasetAdapter


df_companies = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS, "andrewmvd/sp-500-stocks", "sp500_companies.csv"
).drop_duplicates(subset=["Shortname"], keep="first")
companies = df_companies[df_companies["Country"] == "United States"]["Symbol"].values
with open("companies.json", "r", encoding="utf-8") as f:
    data = json.load(f)
missing_companies = [entry["ticker"] for entry in data["missing_companies"]]
find_cik = {entry["ticker"]: entry["cik"] for entry in data["find_cik"]}
companies = [c for c in companies if c not in missing_companies]
search_queries = ["IT capability", "organizational resilience"]
report_year = [2014, 2023]  # 2014/1/1-2023/12/31
define_folder = "define/"
report_folder = "paper_7/report/"
filter_folder = "paper_7/filter/"
if not os.path.exists(report_folder):
    os.makedirs(report_folder)
if not os.path.exists(filter_folder):
    os.makedirs(filter_folder)
for search_query in search_queries:
    folder = filter_folder + search_query + "/"
    if not os.path.exists(folder):
        os.makedirs(folder)
    for x in range(report_year[0], report_year[1] + 1):
        fol = folder + str(x) + "/"
        if not os.path.exists(fol):
            os.makedirs(fol)
for x in range(report_year[0], report_year[1] + 1):
    folder = report_folder + str(x)
    if not os.path.exists(folder):
        os.makedirs(folder)


def save_to_file(content, path):
    with open(path, "w", encoding="utf-8") as file:
        file.write(content)

In [4]:
# keywords
keywords = {}
for search_query in search_queries:
    with open(f"{define_folder}{search_query}.json", "r") as f:
        keywords[search_query] = json.load(f)["keyword"]

In [6]:
# 篩選
log = ""
for search_query in search_queries:
    print(search_query)
    loop = tqdm(range(report_year[0], report_year[1] + 1))
    for year in loop:
        loop.set_description(f"篩選 {year} 年文本")
        report_files = [f for f in os.listdir(report_folder + str(year))]
        for file in report_files:
            input_file = f"{report_folder}/{str(year)}/{file}"
            output_file = f"{filter_folder}{search_query}/{str(year)}/{file}"
            if not os.path.exists(output_file):
                with open(input_file, "r", encoding="utf-8") as f:
                    text = f.read()
                # 分段處理
                paragraphs = text.strip().split("\n")
                # 轉小寫做比對（保留原始段落）
                matched_paragraphs = [
                    para
                    for para in paragraphs
                    if any(k.lower() in para.lower() for k in keywords[search_query])
                ]
                # 合併成新文本
                text = "\n".join(matched_paragraphs)
                save_to_file(text, output_file)
print(log)

IT capability


篩選 2018 年文本:  40%|████      | 4/10 [00:34<00:51,  8.52s/it]


KeyboardInterrupt: 